<a href="https://colab.research.google.com/github/PawelGradek/DO/blob/main/Tensorflow_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1) Install tensorflow 2.7.0**


In [ ]:
!pip install tensorflow==2.7.0


Step to doing: 1,2,3,4,5,13 

# **2) Import liblaries**

In [ ]:
import tensorflow as tf
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd

# **3)  Connect to google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/ /mydrive

!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'   customTF1   customTF2_proba   im   Studia


# **4) Clone TensorFlow models and install TensorFlow Object Detection API**





In [ ]:
!git clone --q https://github.com/tensorflow/models.git

%cd models/research

!protoc object_detection/protos/*.proto --python_out=.

!cp object_detection/packages/tf2/setup.py .  
!python -m pip install .


# **5) Test the model builder**


In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

# **6) unzip images and annotations folders**



In [ ]:
%cd /mydrive/customTF1/data/

!unzip /mydrive/customTF1/images.zip -d .
!unzip /mydrive/customTF1/annotations.zip -d .

# **7) Create test_labels and train_labels**


In [ ]:
!mkdir test_labels train_labels


# (20% of the labels) to the testing dir
!ls annotations/* | sort -R | head -800 | xargs -I{} mv {} test_labels/

!ls annotations/* | xargs -I{} mv {} train_labels/


# **8) Create the CSV files and the "label_map.pbtxt" file**


In [ ]:
def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      classes_names.append(member[0].text)
      value = (root.find('filename').text  ,       
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               int(member[5][0].text),
               int(member[5][2].text),
               int(member[5][1].text),
               int(member[5][3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name) 
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

for label_path in ['train_labels', 'test_labels']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

label_map_path = os.path.join("label_map.pbtxt")
pbtxt_content = ""

for i, class_name in enumerate(classes):
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n}}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)
    print('Successfully created label_map.pbtxt ')

Successfully converted train_labels xml to csv.
Successfully converted test_labels xml to csv.
Successfully created label_map.pbtxt 


# **9) Create train.record and test.record files**




In [ ]:
%cd /mydrive/customTF1/data/

/content/gdrive/My Drive/customTF1/data


In [ ]:
!python /mydrive/customTF1/generate_tfrecord.py train_labels.csv  label_map.pbtxt images/ train.record

!python /mydrive/customTF1/generate_tfrecord.py test_labels.csv  label_map.pbtxt images/ test.record

groups: 100% 3042/3042 [00:07<00:00, 409.44it/s]
Successfully created the TFRecords: /content/gdrive/My Drive/customTF1/data/train.record
groups: 100% 797/797 [00:02<00:00, 384.94it/s]
Successfully created the TFRecords: /content/gdrive/My Drive/customTF1/data/test.record


# **10) Download pre-trained model checkpoint** 



In [ ]:
%cd /content/gdrive/MyDrive/customTF1/data

/content/gdrive/MyDrive/customTF1/data


In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
!tar -xzvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# **12) Train the model**






In [ ]:
%cd /content/models/research/object_detection

/content/models/research/object_detection


In [ ]:
!python model_main_tf2.py --pipeline_config_path=/mydrive/customTF1/data/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config --model_dir=/mydrive/customTF1/training --alsologtostderr

2022-01-15 11:27:38.806543: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0115 11:27:38.810697 140677067999104 mirrored_strategy.py:376] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: None
I0115 11:27:38.817845 140677067999104 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0115 11:27:38.818014 140677067999104 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W0115 11:27:38.847276 140677067999104 deprecation.py:347] From /usr/local/lib/python3.7/dist-packages/object_detection/model_lib_v2.py:564: StrategyBase.experiment

In [ ]:
 !pip uninstall opencv-python-headless==4.5.5.62

In [ ]:
!pip install opencv-python-headless==4.1.2.30

In [ ]:
!python exporter_main_v2.py --trained_checkpoint_dir=/mydrive/customTF1/training --pipeline_config_path=/content/gdrive/MyDrive/customTF1/data/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config --output_directory /mydrive/customTF1/data/inference_graph

2022-01-15 14:55:27.704535: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
W0115 14:55:27.977746 140504787240832 deprecation.py:619] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:464: calling map_fn_v2 (from tensorflow.python.ops.map_fn) with back_prop=False is deprecated and will be removed in a future version.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
W0115 14:55:44.099234 14050478724

In [ ]:
!wget https://freefontsdownload.net/download/160187/arial.zip
!unzip arial.zip -d .

%cd utils/
!sed -i "s/font = ImageFont.truetype('arial.ttf', 24)/font = ImageFont.truetype('arial.ttf', 50)/" visualization_utils.py
%cd ..

In [ ]:
!pip install cv2_plt_imshow

In [ ]:
!pip uninstall opencv-python-headless==4.5.5.62

In [ ]:
!pip install opencv-python-headless==4.1.2.30

In [ ]:
import tensorflow as tf
import time
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
from google.colab.patches import cv2_imshow
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import matplotlib.pyplot as plt

PATH_TO_SAVED_MODEL="/mydrive/customTF1/data/inference_graph/saved_model"

detect_fn=tf.saved_model.load(PATH_TO_SAVED_MODEL)
 
category_index=label_map_util.create_category_index_from_labelmap("/mydrive/customTF1/data/label_map.pbtxt",use_display_name=True)


def image_to_array(path):
    return np.array(Image.open(path))


image_path = "/content/gdrive/MyDrive/im/walking_with_dog/walking_the_dog_287.jpg"


image_np = image_to_array(image_path)


tensor = tf.convert_to_tensor(image_np)
tensor = tensor[tf.newaxis, ...]
detections = detect_fn(tensor)



num_detections = int(detections.pop('num_detections'))


detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}

detections['num_detections'] = num_detections


detections['detection_classes'] = detections['detection_classes'].astype(np.int64)


image_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.4, 
      agnostic_mode=False)
%matplotlib inline

plt.figure(figsize=(6,4), dpi=200)
plt.axis("off")
plt.imshow(image_with_detections)
plt.show()